<a href="https://colab.research.google.com/github/daichira-gif/daichira/blob/main/LangChain_GeminiPro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### APIキーの準備

In [1]:
import pathlib
import textwrap
import google.generativeai as genai
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

# Markdown出力
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [2]:
# APIキーの準備
GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

## 1.生成AIを使わずにPDFファイルを読み込み
- tabula-pyで読み込んでみる

In [3]:
!pip install tabula-py
!pip install jpype1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 7.4 MB/s eta 0:00:00


### PDFファイルのアップロード

In [36]:
# ファイルのアップロードを促進するためのウィジェットをインポート
from google.colab import files
import tabula
import pandas as pd

# PDFファイルをアップロード
uploaded = files.upload()

# アップロードされたファイル名を取得
pdf_file = list(uploaded.keys())[0]

# PDFファイルから表を抽出
tables = tabula.read_pdf(pdf_file, pages='all', multiple_tables=True)

Saving 公共施設白書_資料編pdf.pdf to 公共施設白書_資料編pdf.pdf


In [ ]:
# 読み込んだ表を表示
for i, table in enumerate(tables, 1):
    print(f"Table {i}:\n{table}\n")

### 読み込んだ表の整形

In [17]:
import pandas as pd
from io import StringIO

In [44]:
# 任意のページ（データ）を読み込む
data1 = tables[44]
df = data1

# 1行目のデータをカラム名として設定する
df.columns = df.iloc[0]

# 1行目のデータを削除する
df = df.iloc[1:]

# データフレーム内のすべての列で'\r'を探し、それを除去する
df = df.replace('\r', '', regex=True)

# カラム名から'\r'を探し、それを除去する
df.columns = df.columns.str.replace('\r', '')

# データフレームを表示する
df

,施設名,区名,町名,建築年度,築年数,延床面積(m2),棟数,複合の状況,主たる施設名,主な構造,...,建物所有者,所管局,所管課,管理形態,年間運営日数(日),利用者数合計(人),収入(千円),NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,使用料,利用料,その他,合計,NaN
2,寿山市民センター,小倉北区,大畠,1998,24,669.12,1,NaN,NaN,鉄筋コンクリート造,...,市,小倉北区役所,コミュニティ支援課,直営,293,"23,165",252,0,132,384.0
3,桜丘市民センター,小倉北区,上富野,1977,45,703.23,1,NaN,NaN,鉄筋コンクリート造,...,市,小倉北区役所,コミュニティ支援課,直営,294,"17,507",313,0,78,391.0
4,小倉中央市民センター,小倉北区,京町,1970,52,806.49,3,NaN,NaN,鉄筋コンクリート造,...,市,小倉北区役所,コミュニティ支援課,直営,293,"22,270",390,0,110,500.0
5,南丘市民センター,小倉北区,熊谷,1985,37,969.67,1,NaN,NaN,鉄筋コンクリート造,...,市,小倉北区役所,コミュニティ支援課,直営,297,"24,124",290,0,39,329.0
6,三郎丸市民センター,小倉北区,熊本,1982,40,787.02,2,NaN,NaN,鉄筋コンクリート造,...,市,小倉北区役所,コミュニティ支援課,直営,294,"18,063",301,0,49,350.0
7,霧丘市民センター,小倉北区,黒原,1977,45,728.13,1,NaN,NaN,鉄筋コンクリート造,...,市,小倉北区役所,コミュニティ支援課,直営,298,"54,267",544,0,174,718.0
8,到津市民センター,小倉北区,下到津,2000,22,440.82,1,複合,到津市民センター・こども文化会館,鉄筋コンクリート造,...,市,小倉北区役所,コミュニティ支援課,直営,310,"24,449",281,0,98,379.0
9,中島市民センター,小倉北区,昭和町,2000,22,674.04,1,複合,中島市民センター,鉄筋コンクリート造,...,市,小倉北区役所,コミュニティ支援課,直営,295,"26,520",281,0,159,440.0
10,貴船市民センター,小倉北区,白銀,1977,45,705.00,1,NaN,NaN,鉄筋コンクリート造,...,市,小倉北区役所,コミュニティ支援課,直営,296,"44,701",474,0,96,570.0


# LangChainを使って、Google Geminiを動かしてみる
- 参考URL https://qiita.com/t_kamiya78/items/4fbb802a98a32bf4c5f7

### ライブラリのインストール

In [ ]:
# パッケージのインストール
!apt-get install tesseract-ocr-jpn
!apt-get install libtesseract-dev
!pip install Pillow==9.0.0
!pip install -U --quiet langchain-google-genai pillow
!pip install pytesseract

### LangChainで読み込む

In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-pro', google_api_key = GOOGLE_API_KEY )

df_str = df.to_string()

result = llm.invoke(f'次の表から、湯川市民センターの町名を教えてください。:\n{df_str}')
print(result.content)
# 湯川市民センターの町名は、湯川です。


result = llm.invoke(f'次の表を表示してください。:\n{df_str}')
print(result.content)





湯川市民センターの町名は、湯川です。
| 施設名                                            | 区名 | 町名 | 建築年度 | 築年数 | 延床面積(m2) | 棟数 | 複合の状況 | 主たる施設名 | 主な構造 | 耐震診断 | 耐震補強 | 建物所有者 | 所管局 | 所管課 | 管理形態 | 年間運営日数(日) | 利用者数合計(人) | 収入(千円) | 使用料 | 利用料 | その他 | 合計 |
|----------------------------------------------------|------|------|------------|----------|---------------|------|---------------|--------------|------------|------------|------------|------------|----------|----------|------------|-------------------|----------------------|--------------|----------|----------|----------|----------|
| 寿山市民センター                                  | 小倉北区 | 大畠 | 1998 | 24 | 669.12 | 1 | NaN | NaN | 鉄筋コンクリート造 | 不要 | 不要 | 市 | 小倉北区役所 | コミュニティ支援課 | 直営 | 293 | 23,165 | 384.0 | 252 | 0 | 132 | 384.0 |
| 桜丘市民センター                                  | 小倉北区 | 上富野 | 1977 | 45 | 703.23 | 1 | NaN | NaN | 鉄筋コンクリート造 | 実施済 | 不要 | 市 | 小倉北区役所 | コミュニティ支援課 | 直営 | 294 | 17,507 | 391.0 | 313 | 0 | 78 | 391.0 |
| 小倉中央市民センター                                |